In [ ]:
%AddDeps org.jsoup jsoup 1.11.3

In [ ]:
spark

In [ ]:
// Import mutable collections
import scala.io.Source
import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.ListBuffer
import scala.collection.mutable.Map

In [ ]:
// import jsoup
import org.jsoup.Jsoup
import org.jsoup.nodes.Element
import scala.collection.JavaConversions._

In [ ]:
// Import Java utils for gzip decompression
// Import Java utils for reading files
import java.io.File
import java.io.{FileInputStream}
import java.util.zip.{GZIPInputStream}
import scala.util.Try

In [ ]:
// Generate a list of files
var listFiles= new File("Data/")
val rows = spark.sparkContext.parallelize(listFiles.listFiles.filter(_.isFile).map(_.toString).toSeq)

In [ ]:
// Decompression function for each file
def decompress(compressed: String) = {
    val inputStream = new GZIPInputStream(new FileInputStream(compressed))
    scala.io.Source.fromInputStream(inputStream).mkString
}

In [ ]:
// html object processor
object HTML_X {
    
    def makeDataSet(list_files:org.apache.spark.rdd.RDD[String]) = {

        val mytitles:ListBuffer[String] = ListBuffer.empty[String]
        val mydomains:ListBuffer[String] = ListBuffer.empty[String]
//         val rowsSampleList = list_files.collect().toList

        list_files.collect().foreach(element =>{
            try {
                mytitles += Jsoup.parse(decompress(compressed = element))
                        .getElementsByTag("title").text()
                mydomains += element.toString.split("_")(0).split("/")(1)
            } catch {
                case e: Exception =>{
                    println("Corrupted file exception %s".format(element))
         }}
            

        })
    //     var rowsDataTitles = mydomains.toList zip mytitles.toList
        (mydomains.toList,mytitles.toList).zipped.toSeq.toDF("domain","title")

}
}

In [ ]:
class PreProcessor {
    
    def Remover{
        
//         remove comments
    }
        
}

In [ ]:
val rowsSample = rows.sample(withReplacement=false,
                             fraction = 0.3, 
                             seed = 29)
rowsSample.count()

In [ ]:
val article_html = Jsoup.parse(decompress(compressed = rowsSample.takeSample(withReplacement = false,
                                                                             num = 1, 
                                                                             seed = 29)(0)))

In [ ]:
val links = article_html.select("a,h1,h2,p")

In [ ]:
import scala.util.control.Breaks._

In [ ]:
var i = 0
for (element <- links){
    println(element.toString)
    println(element.text())
    i = i + 1;
    if (i ==50){break}
}

In [ ]:
article_html.remove("head")

In [ ]:
var dataTitles = HTML_X.makeDataSet(list_files=rowsSample)

In [ ]:
dataTitles.show(100)

In [ ]:
dataTitles.createTempView("hg")

In [ ]:
dataTitles.count()

In [ ]:
%%SQL

SELECT domain, COUNT(title) as total_news
FROM hg
GROUP BY domain
ORDER BY total_news DESC
LIMIT 100

In [ ]:
import org.apache.spark.sql.functions


In [ ]:
import org.apache.spark.sql.functions.{lit, ltrim, rtrim, rpad, lpad, trim}

var titles_d = dataTitles.select(functions.col("title")).take(20).